# Libraries

In [2]:
import re
import tqdm
import spacy
import torch
import random
import torch.nn as nn
import torch.optim as optim
import numpy as np
import scipy.sparse
import pandas as pd
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import nltk
from nltk.corpus import stopwords


In [3]:
STOPWORDS=stopwords.words('english')

# Parsing new dataset

In [4]:
def handle_apos(text):
    text = re.sub("'re", " are", text)
    text = re.sub("'m", " am", text)
    text = re.sub("'ll", " will", text)
    text = re.sub("'ve", " have", text)
    text = re.sub("'d", " would", text)
    text = re.sub("'t", " not", text)
    return text

def handle_punctuation(text):
    text = text.replace(".", " ")
    text = text.replace(",", " ")
    text = text.replace("!", " ")
    text = text.replace("?", " ")
    text = text.replace(";", " ")
    text = text.replace(":", " ")
    text = text.replace("-", " ")
    text = text.replace("(", " ")
    text = text.replace(")", " ")
    text = text.replace("[", " ")
    text = text.replace("]", " ")
    text = text.replace("{", " ")
    text = text.replace("}", " ")
    text = text.replace("/", " ")
    text = text.replace("\\", " ")
    text = text.replace("*", " ")
    text = text.replace("+", " ")
    text = text.replace("=", " ")
    text = text.replace("#", " ")
    text = text.replace("%", " ")
    text = text.replace("$", " ")
    text = text.replace("@", " ")
    text = text.replace("&", " ")
    text = text.replace("^", " ")
    text = text.replace("|", " ")
    text = text.replace("~", " ")
    text = text.replace("`", " ")
    text = text.replace("'", "")
    text = text.replace("\"", " ")
    text = text.replace("<", " ")
    text = text.replace(">", " ")
    text = text.replace("  ", " ")
    return text

In [5]:
def stop_words(text):
    text=text.split()
    re=[]
    for x in text:
        if x not in STOPWORDS:
            re.append(x)
    return " ".join(re)

# Tfidf Implementation

In [23]:
df = pd.read_csv('../input/domaintermextraction/data_sw.csv')
df.head()


,content,category
0,quarterly profits us media giant timewarner ju...,business
1,owners embattled russian oil giant yukos ask b...,business
2,british airways blamed high fuel prices drop p...,business
3,shares uk drinks food firm allied domecq risen...,business
4,japans economy teetered brink technical recess...,business


In [24]:
data={}
arr=["business","tech","sport","politics","entertainment"]
for i in arr:
    data[i]=np.array(df[df["category"]==i]["content"])

In [40]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    sorted_items = sorted_items[:topn]
    score_vals = []
    feature_vals = []
    for idx, score in sorted_items:
        fname = feature_names[idx]
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    return results

def get_top_nK_words(corpus, K=1, n=None):
    vec1 = CountVectorizer(stop_words=STOPWORDS, ngram_range=(K,K)).fit(corpus)
    bag_of_words = vec1.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in     
                  vec1.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], 
                reverse=True)
    temp=words_freq[:n]
    dic={}
    for i in temp:
        dic[i[0]]=i[1]
    return dic
    

In [45]:
label=[]
ngram=[]
keywd=[]
freqq=[]
alpha=[]
K_arr=[1,2,3]
K=3  # n-gram 
for key_lb in arr:
    cv=CountVectorizer(stop_words=STOPWORDS,ngram_range=(K,K))
    word_count_vector=cv.fit_transform(data[key_lb].tolist())
    word_count_vector.shape
    tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
    tfidf_transformer.fit(word_count_vector)
    docs=data[key_lb].tolist()
    unigram=[]
    for i in range(data[key_lb].shape[0]):
        n=10
        feature_names=cv.get_feature_names()
        doc=docs[i]
        tf_idf_vector=tfidf_transformer.transform(cv.transform([doc]))
        sorted_items=sort_coo(tf_idf_vector.tocoo())
        keywords=extract_topn_from_vector(feature_names,sorted_items,n)
        for k in keywords:
            unigram.append([k,keywords[k]])
    unigram.sort(key=lambda y:y[1], reverse=True)
    val=get_top_nK_words(data[key_lb],K)
    for j in unigram:
        label.append(key_lb)
        ngram.append(str(K)+"-gram")
        keywd.append(j[0])
        freqq.append(val[j[0]])
        alpha.append(j[1])


/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [46]:
data_f = {'n-gram': ngram, 'domain': label,'keyword':keywd,'frequency':freqq,'tfidf-score':alpha}
df = pd.DataFrame(data_f)
df.head()

,n-gram,domain,keyword,frequency,tfidf-score
0,3-gram,business,bn rupees bn,8,0.269
1,3-gram,business,standard chartered said,5,0.245
2,3-gram,business,mr spitzers office,3,0.226
3,3-gram,business,financial ombudsman service,3,0.218
4,3-gram,business,ms mohd saaid,3,0.217


In [47]:
df.to_csv('./tfidf_trigram_fre.csv')  